# 3. Знакомимся с новыми данными: коронавирус

Мы будем изучать методы визуализации, используя два набора данных:
1) COVID_DATA - данные о показателях заболеваемости за каждый день в различных регионах 187 стран.
    Мы будем работать со следующими столбцами:
    - date - дата наблюдения;
    - province/state - наименование провинции/штата;
    - country - наименование страны;
    - confirmed - общее число зафиксированных случаев на указанный день;
    - deaths - общее число зафиксированных смертей на указанный день;
    - recovered - общее число выздоровлений на указанный день.

In [2]:
import pandas as pd
covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


2) country_vaccinations - данные о процессе вакцинирования лидей в 221 стране мира. 

        Данная таблица содержит следующие столбцы:
        - country - наименование страны;
        - date - дата наблюдения;
        - total_caccinations - общее чисор ввеленных вакцин в стране на указанный день;
        - people_vaccinated - общее число привитых первыми компонентом в стране на указанный день;
        - people_fully_vaccinated - общее число привитыз вторым компонентом в тране на указанный день (первый компонент уже был введен им ранее);
        - people_fully_vaccinated_per_hundred - процент привитых вторым компонентом в стране на указанный день;
        - daily_vaccination - ежедневная вакцинация (число вакцинированных в указанный день);
        - vaccines - комбинации вакцин, используемые в стране.

In [3]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
    ]

display(vaccinations_data.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


### НЕМНОГО ОБ ОСОБЕННОСТЯХ ДАННЫХ

В таблице covid_data данные о заболеваемости представлены для каждого региона (штата) страны за каждый день. Например, для США записи за 29 мая 2021 года имеют вид из нескольких строк.

В таблице vaccinations_data параметры вакцинирования отображены для всех стран на каждый день, без учета регионов. Вот часть таблицы для тех же США за 29 мая 2021 года. И в ней всего одна строка.

В данных много пропусков, а также неверно введенных значений. Например, лля США и Великобритании в большинстве записей таблицы covid_data число выздровевших людей равно 0. В данном моделе мы закроем на это глаза, но вообще эти страны полдежали бы исключению из данных или нам понадобился бы дополнительный источник данных, из которого иы смогли бы получить недостающую информацию.

ОБРАТИТЕ ВНИМАНИЕ, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated - это сумарные показатели по стране, то ест с кадлым днем они должны расти. Такие признаки называют НАКОПИТЕЛЬНЫМИ.

Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:
1) В таблице covid_data необходимо предвартьельно рассчитать суммарное ежедневное число заболевшиз во всез провинциях/штатах в каждой стране.
2) В таблицах не воспадает число стран, а иногда и из названия.
    При объединении таблиц по столбцу мы определенно теряем данные (в данной задаче потери незначительны). Избежать этого можно ручными преобразованиями данных - искать различи в названиях стран в таблицах и преобразовывать из. Однако это не является темой данного модуля. 
3) Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерть большое количество наблюдений в таблице covid_data.

### ПРЕДОБРАБОТКА ДАННЫХ

Опираясь на замечания выше, выполним небольшую предобработку.


В таблице covid_data:
    
    1) Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регонам к данныи по странам.

In [4]:
covid_data = covid_data.groupby(
    ['date', 'country'],
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

    2) Преобразуем даты в формат datetime с помощью функции pd.to_datetime():

In [5]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

    3) Создадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число сметртей и число выздоровевших пациентов:

In [6]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

    4) Создадим признак ежедневного прироста число заболевших, умерщих и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведем группировку по странам и рассчитаем разницу между "вчера и сегодня" с помощью метода diff():

In [7]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

Итоговый вид таблицы covid_data:

In [8]:
covid_data.head()

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
11337,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
11570,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11807,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12051,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12299,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime:

In [9]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

Задание 3.1

За какой период представлены данные в таблице covid_data? В качестве ответа введите даты в формате datetime (без указания времени).

ОТВЕТ 2020-01-22-2021-05-29

In [10]:
# задание 3.1
print(f"Данные о заболеваемости представлены от {covid_data['date'].min()} до {covid_data['date'].max()}")

Данные о заболеваемости представлены от 2020-01-22 00:00:00 до 2021-05-29 00:00:00


Задание 3.2

За какой период представлены данные в таблице vaccinations_data? В качестве ответа введите даты в формате datetime без указания времени.
Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД

ОТВЕТ:

In [11]:
# Задание 3.2

print(f"Данные о вакцинированных представлены от {vaccinations_data['date'].min()} до {vaccinations_data['date'].max()}")

Данные о вакцинированных представлены от 2020-12-02 00:00:00 до 2021-09-06 00:00:00


Задание 3.3

С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.
Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1. То есть в результирующую таблицу должны попасть все записи из таблицы covid_data и из её пересечения с vaccinations_data, но не более. Результат объединения занесите в переменную covid_df.
Сохраните таблицу covid_df — она понадобится нам в следующем юните.

Сколько строк и столбцов в таблице covid_df?
Введите ответ в виде двух чисел через дефис (например, 333-33): первое число — количество строк, второе число — количество столбцов.

ОТВЕТ: 86785-16

In [12]:
covid_df = covid_data.merge(
    vaccinations_data,
    on=['date','country'],
    how= 'left'
)
display(covid_df.head())
covid_df.shape

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(86785, 16)

В получившейся в задании 3.3 таблице covid_df создайте признаки death_rate — общий процент смертей среди зафиксированных случаев (летальность) и recover_rate — общий процент случаев выздоровления. Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered) к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%.

In [13]:
covid_df['death_rate'] = covid_df['deaths'] / covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered'] / covid_df['confirmed'] * 100


Задание 3.4

Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой.

ОТВЕТ: 7.19

In [14]:
mask = covid_df['country'] == 'United States'
round(covid_df[mask]['death_rate'].max(),2)

7.19

Задание 3.5

Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.

ОТВЕТ: 

In [15]:
mask1 = covid_df['country'] == 'Russia'
round(covid_df[mask1]['recover_rate'].mean(),2)

67.06

In [16]:
covid_df.to_csv('data/covid_df.csv')